In [191]:
#run_name = "matteo_?_cumulative*"
#run_name = "matteo_?laps*"
#run_name = "rl_trained*"
#run_name = "matte_8laps_aio"
result_path = "../Training/results"
training_step_length = 100_000

In [179]:
import glob
import json
import matplotlib.pyplot as plt
import os
from tensorflow.python.summary.summary_iterator import summary_iterator
import numpy as np

In [198]:
def generate_plotdict(evallist: list) -> dict:
    plotdict = {}
    for eval in evallist:
        with open(eval) as e:
            jdict = json.load(e)
            trackName = jdict["trackName"]
            if trackName not in plotdict:
                plotdict[trackName] = list()
            plotdict[trackName].append(jdict["result"])
    return plotdict

def plot_dict(plotdict: dict, color_dict, artist = None):
    if artist == None:
        artist = plt.subplot()
    if all(len(v) == 1 for v in plotdict.values()):
        artist.bar(list(plotdict.keys()), [v[0] for v in plotdict.values()], color=[color_dict[k] for k in plotdict.keys()])
    else:
        for track in plotdict.keys():
            #artist.plot(range(training_step_length, (len(plotdict[track]) + 1) * training_step_length, training_step_length), plotdict[track], label=track, color=color_dict[track])
            artist.plot(range(2,11), plotdict[track], label=track, color=color_dict[track])
        artist.legend()
    artist.set_ylim(0, 1)
    artist.set_title("Evaluation")

def extract_tfevents(path: str, tag: str = "Policy/Gail Reward"):
    plotlist = []
    tffile = glob.glob(f"{path}/Kart/events*")[0]
    for e in summary_iterator(tffile):
        for v in e.summary.value:
            if v.tag == tag:
                plotlist.append(v.simple_value)
    return plotlist

# SUPER EVALUATION-O-MATIC 9000

In [181]:
track_list = ["Square", "8", "Monza", "Spa", "Twisty"]
color_list = [f"C{n}" for n in range(5)]
color_dict = {track_list[n]: color_list[n] for n in range(5)}

In [ ]:
tag = "Policy/Gail Reward"

for trackN in range(5):
    evallist = []
    tflist = []
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    for run in glob.glob(f"{result_path}/{run_name}_Track{trackN}_*"):
        evallist.append(run + "/" + glob.glob("evaluation*.json", root_dir=run)[0])
        tflist += extract_tfevents(run, tag)    
    
    plot_dict(generate_plotdict(evallist), ax2, color_dict)
    ax1.plot(np.linspace(1e5, 1e6, len(tflist)), tflist, label= track_list[trackN], color=color_list[trackN])
    ax1.legend()
    ax1.set_ylim(0, 1000)
    ax1.set_title(tag)
    fig.suptitle(run_name)
    plt.show()

In [ ]:
tag = "Policy/Gail Reward"

evallist = []
ax1 = plt.subplot(223)
ax2 = plt.subplot(224)

for trackN in range(4):
    trackName = track_list[[0, 1, 2, 4][trackN]]
    tflist = []

    axn = plt.subplot(2, 4, trackN + 1)

    for run in glob.glob(f"{result_path}/{run_name}_{trackN}"):
        evallist.append(run + "/" + glob.glob("evaluation*.json", root_dir=run)[0])
        tflist += extract_tfevents(run, tag)    
    axn.plot(np.linspace(1e5, 1e6, len(tflist)), tflist, label=trackName, color=color_dict[trackName])
    axn.legend()
    axn.set_ylim(0, 1200)
    #axn.set_aspect(1000.0)
    #axn.set_title(tag)

plotdict = generate_plotdict(evallist)
ax1.bar(range(4), plotdict["Spa"], label="Spa", color=color_dict["Spa"])
ax2.bar(range(4), plotdict["Spa"], label="Spa", color=color_dict["Spa"])
ax1.set_ylim(0, 1)
ax2.set_ylim(0.55, 0.75)
ax1.legend()
#ax1.set_title("Evaluation")

ax1.indicate_inset_zoom(ax2)
plt.suptitle(run_name)
#plt.tight_layout()
plt.show()

# Evaluation Plot

The following cell loads all `evaluation<...>.json` files and plots their values.

In [ ]:
evallist = glob.glob(f"{result_path}/{run_name}/evaluation*.json")

plotdict = generate_plotdict(evallist)

plot_dict(plotdict, color_dict=color_dict)

plt.show()

# Final Evaluation Plot

The following cell takes the last elements for each `run_name<...>_N` series of runs and plots their `evaluation<...>.json` files.

In [ ]:
all_evaluations = glob.glob(f"{result_path}/{run_name}/evaluation*.json")

all_runs = list(map(lambda evaljson: os.path.dirname(evaljson), all_evaluations))
highest_run_number = max(map(lambda run: run.split('_')[-1], all_runs))
all_last_runs = list(filter(lambda run: run.endswith(highest_run_number), all_runs))
evallist = list(map(lambda run: os.path.join(run, glob.glob("evaluation*.json", root_dir=run)[0]), all_last_runs))

#print(all_last_runs, end="\n")
plotdict = generate_plotdict(evallist)

plot_dict(plotdict, color_dict=color_dict)

plt.show()